制約付きMDP

参考

* [Learning with Safety Constraints: Sample Complexity of Reinforcement Learning for Constrained MDPs](https://arxiv.org/abs/2008.00311)

強化学習では累積報酬和を最大化するような方策を学習することが目標ですが、実際のアプリケーションでは方策の挙動に対して何らかの制約が求められる場合があります。
制約付きMDP（CMDP）では、このような制約を満たしながら累積報酬和を最大化することを目指します。

表記

* MDP: $M=\left( S, A, P, r, c, \bar{C}, s_0, H\right)$
    1. 有限状態集合: $S=\{1, \dots, |S|\}$
    2. 有限行動集合: $A=\{1, \dots, |A|\}$
    3. 遷移確率行列: $P\in \mathbb{R}^{SA\times S}$
    4. 報酬行列: $r\in [0, 1]^{S\times A}$
    5. $N$個のコスト関数：
        1. $c(i, s, a) \in [0, 1]$は$i \in \{1,\dots, N\}$番目の制約によって与えられるコスト関数
        2. $\bar{C}$は満たすべき制約の値。$i \in \{1,\dots, N\}$について、$\bar{C}_i\in [0, \bar{C}_{\max}]$
    6. 初期状態: $s_0$
    7. ホライゾン: $H$
* $a \sim \pi(s, \cdot, h)$：$h$ステップ目で方策が出す行動
* $P_\pi^h f(s)=P_\pi P_\pi \ldots P_\pi f$


価値関数：
$$
V_t^\pi(s)=\mathbb{E}\left[\sum_{h=t}^{H-1} r\left(s_h, a_h\right) ; a_h \sim \pi\left(s_h, \cdot, h\right), s_t=s\right]
$$

価値の分散：
$$
\sigma_h^{\pi^2}(s)=\mathbb{E}\left[\left(V_{h+1}^\pi\left(s_{h+1}\right)-P_\pi V_{h+1}^\pi(s)\right)^2\right]
$$

$i$番目の制約の価値：
$$
C_{i, t}^\pi(s)=\mathbb{E}\left[\sum_{h=t}^{H-1} c\left(i, s_h, a_h\right) ; a_t \sim \pi\left(s_h, \cdot, h\right), s_t=s\right]
$$

有限ホライゾンのCMDPの問題は次で定義されます

$$
\max _\pi V_0^\pi\left(s_0\right) \text { s.t. } C_{i, 0}^\pi\left(s_0\right) \leq \bar{C}_i, \quad \forall i \in\{1, \ldots, N\}
$$

以下ではこの問題には実行可能解があるとして、$V_0^*\left(s_0\right)=V_0^{\pi^*}\left(s_0\right)$と書きます。
また、次の意味でPACなアルゴリズムを探します：

---

**定義**
アルゴリズム$\mathcal{A}$について、$\mathcal{A}$が次を達成する際に必要なサンプル数をサンプル効率と呼ぶ：

$$
\begin{aligned}
\mathbb{P}\left(V_0^{\pi^*}\left(s_0\right) - V_0^{\mathcal{A}}\left(s_0\right) \leq \epsilon \text { and } 
C_{i, 0}^{\mathcal{A}}\left(s_0\right) - \bar{C}_i \leq \epsilon \; \forall i \in\{1, \ldots, N\}\right) \geq 1-\delta
\end{aligned}
$$

---

## Optimistic-GMBL アルゴリズム

まずはGenerative modelを使ったアルゴリズムについて見てみましょう。
以下のOptimistic Generative Model Based Learning アルゴリズムについて解析していきます。
各状態行動について一定数のデータをサンプルし、続いてモデルベースでMDPを構築、最後に線型計画法でCMDPを解きます。

\begin{equation}
\begin{aligned}
& \text { 1: Input: accuracy } \epsilon \text { and failure tolerance } \delta \text {. } \\
& \text { 2: Set } \delta_P=\frac{\delta}{12(N+2)|S|^2|A| H} \text {. } \\
& \text { 3: Set } n\left(s^{\prime}, s, a\right)=0 \forall\left(s, a, s^{\prime}\right) \text {. } \\
& \text { 4: for each }(s, a) \in S \times A \text { do } \\
& \text { 5: } \quad \text{ Sample } (s, a), n=\frac{256}{\epsilon^2}|S| H^3 \log \frac{12(N+2)|S||A| H}{\delta} \text { and update } n\left(s^{\prime}, s, a\right) \text {. } \\
& \text { 6: } \quad \widehat{P}\left(s^{\prime} \mid s, a\right)=\frac{n\left(s^{\prime}, s, a\right)}{n} \forall s^{\prime} \text {. } \\
& \text { 7: Construct } \mathcal{M}_{\delta_P} \text { according to (5). } \\
& \text { 8: Output } \tilde{\pi}=\operatorname{ELP}\left(\mathcal{M}_{\delta_P}\right) \text {. }
\end{aligned}
\end{equation}

Optimistic-GMBLは各$(s, a)$で次状態$s'$をサンプルし、$\widehat{P}\left(s^{\prime} \mid s, a\right)=\frac{n\left(s^{\prime}, s, a\right)}{n} \forall\left(s^{\prime}, s, a\right)$を構築します。
このとき、Optimistic-GMBLは次のようにMDP $M_{\delta_P}$ を構築します（$P$がある程度の範囲を持ってます）：

$$
\begin{aligned}
& \mathcal{M}_{\delta_P}:=\left\{M^{\prime}: r^{\prime}(s, a)=r(s, a),
c^{\prime}(i, s, a)=c(i, s, a), H^{\prime}=H, s_0^{\prime}=s_0\right. \\
& \left|P^{\prime}\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq 
\min \left(\sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n} \log \frac{4}{\delta_P}}+\frac{2}{3 n} \log \frac{4}{\delta_P},
\left.\sqrt{\frac{\log 4 / \delta_P}{2 n}}\right) \forall s, a, s^{\prime}, i\right\},
\end{aligned}
$$

最後のステップではOptimistic-GMBLは次のCMDPを解きます：
$$
\max _{\pi, M^{\prime} \in \mathcal{M}_{\delta_P}} V_0^{\prime \pi}\left(s_0\right) \quad \text { s.t. } \quad C_{i, 0}^{\prime \pi}\left(s_0\right) \leq \bar{C}_i \forall i
$$

これは一番大きい$M'$を解いている点でOptimisticです。

---

**なぜこんなことをするのか？**

雑に$P$を点推定すると、実行不可能な$M'$が選択される可能性があります。
これを解決するために、真のMDPが高い確率で$M_{\delta_P}$に入っているようにします。

---

次のサンプル効率を導出していきましょう. 

---

$\epsilon \in\left(0, \frac{2}{9} \sqrt{\frac{H}{|S|}}\right)$かつ$\delta\in (0, 1)$のとき、アルゴリズム$\mathcal{A}$が$\tilde{M}=\left\langle S, A, \tilde{P}, r, c, \bar{C}, s_0, H\right\rangle$と方策$\tilde{\pi}$を吐き出すとき、それは
$$
\mathbb{P}\left(V_0^{\tilde{\pi}}\left(s_0\right) \geq V_0^{\pi^*}\left(s_0\right)-\epsilon \text { and } C_{i, 0}^{\tilde{\pi}}\left(s_0\right) \leq \bar{C}_i+\epsilon \forall i \in\{1,2, \ldots, N\}\right) \geq 1-\delta
$$
を満たし、必要なサンプル数は$\frac{256}{\epsilon^2}|S|^2|A| H^3 \log \frac{12(N+2)|S||A| H}{\delta}$である。

---


### 証明

以下、$\tilde{M}$での価値関数を$\tilde{V}$とします。

**ステップ１** 高い確率で真のMDPを含むことを示します。つまり、$\mathbb{P}\left(M \in \mathcal{M}_{\delta_P}\right) \geq 1-|S|^2|A| \delta_P$を示します。

全ての状態行動で$n$個以上のサンプルがあるとします。
まず、Hoeffdingの不等式より

$$
\mathbb{P}\left(\left|P\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{\log 4 / \delta_P}{2 n}}\right) \geq 1-\delta_P / 2
$$

が成り立ちます。続いて、Bernsteinの不等式より
$$
\mathbb{P}\left(\left|P\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n} \log \frac{4}{\delta_P}}+\frac{2}{3 n} \log \frac{4}{\delta_P}\right) \geq 1-\delta_P / 2
$$

も成立します。これを組み合わせて、

$$
\mathbb{P}\left(\left|P\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \min \left\{\sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n} \log \frac{4}{\delta_P}}+\frac{2}{3 n} \log \frac{4}{\delta_P}, \sqrt{\frac{\log 4 / \delta_P}{2 n}}\right\}\right) \geq 1-\delta_P
$$

が成り立ちます。あとはUnion Boundを取るだけです。

**ステップ２** $\mathcal{M}_{\delta_P}$に含まれる遷移確率同士はそれなりに近いことを示します。
$p, \widehat{p}, \tilde{p} \in[0,1]$ として、
$$
\mathcal{P}_{\delta_P}:=\left\{p^{\prime} \in[0,1]:\left|p^{\prime}-\widehat{p}\right| \leq \min \left(\sqrt{\frac{2 \widehat{p}(1-\widehat{p})}{n} \log 4 / \delta_P}+\frac{2}{3 n} \log 4 / \delta_P, \sqrt{\frac{\log 4 / \delta_P}{2 n}}\right)\right\}
$$
とします。また、$\mathbb{P}\left(p \in \mathcal{P}_{\delta_P}\right) \geq 1-\delta_P$ と $\tilde{p} \in \mathcal{P}_{\delta_P}$を仮定します。
このとき、確率$1-\delta_P$以上で

$$
|p-\tilde{p}| \leq \sqrt{\frac{8 \tilde{p}(1-\tilde{p})}{n} \log 4 / \delta_P}+2 \sqrt{2}\left(\frac{\log 4 / \delta_P}{n}\right)^{\frac{3}{4}}+3 \sqrt{2} \frac{\log 4 / \delta_P}{n}
$$

を証明します。$p$が高い確率で$\mathcal{P}_{\delta_P}$に、$\tilde{p}$は常に$\mathcal{P}_{\delta_P}$に入ってることを踏まえると、これはそんなに難しくありません：

$$
\begin{aligned}
|p-\tilde{p}| & \leq|p-\widehat{p}|+|\widehat{p}-\tilde{p}| \leq 2 \sqrt{\frac{2 \widehat{p}(1-\widehat{p})}{n} \log 4 / \delta_P}+\frac{4}{3 n} \log 4 / \delta_P \\
& \leq 2 \sqrt{\frac{2 \log 4 / \delta_P}{n}\left(\tilde{p}+\sqrt{\frac{\log 4 / \delta_P}{2 n}}\right)\left(1-\tilde{p}+\sqrt{\frac{\log 4 / \delta_P}{2 n}}\right)}+\frac{4}{3 n} \log 4 / \delta_P \\
& =2 \sqrt{\frac{2 \log 4 / \delta_P}{n}\left(\tilde{p}(1-\tilde{p})+\sqrt{\frac{\log 4 / \delta_P}{2 n}}+\frac{\log 4 / \delta_P}{2 n}\right)}+\frac{4}{3 n} \log 4 / \delta_P \\
& \leq \sqrt{\frac{8 \tilde{p}(1-\tilde{p})}{n} \log 4 / \delta_P}+2 \sqrt{2}\left(\frac{\log 4 / \delta_P}{n}\right)^{\frac{3}{4}}+3 \sqrt{2} \frac{\log 4 / \delta_P}{n} .
\end{aligned}
$$

**ステップ３**

任意の方策$\pi$について、
$$
V_0^\pi-V_0^{\prime \pi}=\sum_{h=0}^{H-2} P_\pi^{\prime h-1}\left(P_\pi-P_\pi^{\prime}\right) V_{h+1}^\pi \text { and } V_0^\pi-V_0^{\prime \pi}=\sum_{h=0}^{H-2} P_\pi^{h-1}\left(P_\pi-P_\pi^{\prime}\right) V_{h+}^{\prime \pi}
$$
と
$$
C_{i, 0}^\pi-C_{i, 0}^{\prime \pi}=\sum_{h=0}^{H-2} P_\pi^{\prime h-1}\left(P_\pi-P_\pi^{\prime}\right) C_{i, h+1}^\pi \text { and } C_{i, 0}^\pi-C_{i, 0}^{\prime \pi}=\sum_{h=0}^{H-2} P_\pi^{h-1}\left(P_\pi-P_\pi^{\prime}\right) C_{i, h+1}^{\prime \pi}
$$

が成り立つことを示します。これも簡単です：

$$
\begin{aligned}
& V_h^\pi(s)-V_h^{\prime \pi}(s)=r_\pi(s)+\sum_{s^{\prime}} P_\pi\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)-\left(r_\pi(s)+\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right)\right) \\
& =\sum_{s^{\prime}} P_\pi\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)-\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)+\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)-\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right) \\
& =\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^\pi\left(s^{\prime}\right)+\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right)\left(V_{h+1}^\pi\left(s^{\prime}\right)-V_{h+1}^{\prime \pi}\left(s^{\prime}\right)\right) .
\end{aligned}
$$

であり、これを再帰的に繰り返すだけです。

**ステップ４**

$$
\left|P\left(s^{\prime} \mid s, a\right)-P^{\prime}\left(s^{\prime} \mid s, a\right)\right| \leq c_1+c_2 \sqrt{P^{\prime}\left(s^{\prime} \mid s, a\right)-\left(1-P^{\prime}\left(s^{\prime} \mid s, a\right)\right)}
$$

が確率$1-\delta_P$以上で成り立つとき、任意の方策$\pi$について、
$$
\left|\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right)\right| \leq|S| c_1\left\|V_{h+1}^{\prime \pi}\right\|_{\infty}+c_2 \sqrt{|S|} \sigma_h^{\prime \pi}(s)
$$
が確率$1-|S|\delta_P$以上で成り立ち、
$$
\left|\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) C_{i, h+1}^{\prime \pi}\left(s^{\prime}\right)\right| \leq|S| c_1\left\|C_{i, h+1}^{\prime \pi}\right\|_{\infty}+c_2 \sqrt{|S|} \sigma_{i, h}^{\prime \pi}(s)
$$
も確率$1-|S|\delta_P$以上で成り立つ。

証明しましょう。
まず、
$$\bar{V}^\pi=\sum_{s^{\prime \prime}} P_\pi^{\prime}\left(s^{\prime \prime} \mid s\right) V_{h+1}^{\prime \pi}\left(s^{\prime \prime}\right)$$
とします。これはつまり$s$の次状態で$V_{h+1}^{\prime \pi}$の期待値を取ったやつです。定数なので、
$$\bar{V}^\pi=\sum_{s^{\prime \prime}} P_\pi^{\prime}\left(s^{\prime \prime} \mid s\right) \bar{V}^\pi=\sum_{s^{\prime \prime}} P_\pi\left(s^{\prime \prime} \mid s\right) \bar{V}^\pi$$
が成り立ちます。これを使って、

$$
\begin{aligned}
& \left|\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right)\right|
=\left|\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right)+\bar{V}^\pi-\bar{V}^\pi\right| \\
& =\left|\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right)\left(V_{h+1}^{\prime \pi}\left(s^{\prime}\right)-\bar{V}^\pi\right)\right| \\
& \leq \sum_{s^{\prime}}\left|P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right|\left|V_{h+1}^\pi\left(s^{\prime}\right)-\bar{V}^\pi\right| \\
& \leq \sum_{s^{\prime}}\left(c_1+c_2 \sqrt{P_\pi^{\prime}\left(s^{\prime} \mid s\right)-\left(1-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right)}\right)\left|V_{h+1}^\pi\left(s^{\prime}\right)-\bar{V}^\pi\right| \\
& \leq|S| c_1\left\|V_{h+1}^{\prime \pi}\right\|_{\infty}+c_2 \sum_{s^{\prime}} \sqrt{P_\pi^{\prime}\left(s^{\prime} \mid s\right)\left(1-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right)\left(V_{h+1}^\pi\left(s^{\prime}\right)-\bar{V}^\pi\right)^2} \\
& \leq|S| c_1\left\|V_{h+1}^{\prime \pi}\right\|_{\infty}+c_2 \sqrt{|S| \sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right)\left(1-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right)\left(V_{h+1}^\pi\left(s^{\prime}\right)-\bar{V}^\pi\right)^2} \\
& \leq|S| c_1\left\|V_{h+1}^{\prime \pi}\right\|_{\infty}+c_2 \sqrt{|S| \sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right)\left(V_{h+1}^\pi\left(s^{\prime}\right)-\bar{V}^\pi\right)^2} \\
& =|S| c_1\left\|V_{h+1}^{\prime \pi}\right\|_{\infty}+c_2 \sqrt{|S|} \sigma_h^{\prime \pi} .
\end{aligned}
$$

**ステップ５**

$\left|P\left(s^{\prime} \mid s, a\right)-P^{\prime}\left(s^{\prime} \mid s, a\right)\right| \leq \frac{a}{\sqrt{n}}$が確率$1-\delta_P$以上で成立しているとき、任意の方策$\pi$について、

$$
\left\|V_{H-1}^\pi-V_{H-1}^{\prime \pi}\right\|_{\infty} \leq \cdots \leq\left\|V_0^\pi-V_0^{\prime \pi}\right\|_{\infty} \leq|S| H^2 a \frac{1}{\sqrt{n}}
$$

が確率$1-|S|^2|A| H \delta_P$以上で成立し、

$$\left\|C_{i, H-1}^\pi-C_{i, H-1}^{\prime \pi}\right\|_{\infty} \leq \cdots \leq\left\|C_{i, 0}^\pi-C_{i, 0}^{\prime \pi}\right\|_{\infty} \leq|S| H^2 a \frac{1}{\sqrt{n}}$$
も確率$1-|S|^2|A| H \delta_P$以上で成立する。

さらに、$n \geq a|S| H^2$の場合、

$$
\left\|\sigma_h^\pi-\sigma_h^{\prime \pi}\right\|_{\infty} \leq \frac{2 \sqrt{2|S| H^2 a}}{n^{1 / 4}}
$$

が確率$1-|S|^2|A| H \delta_P$以上で成立し、

$$
\left\|\sigma_{i, h}^\pi-\sigma_{i, h}^{\prime \pi}\right\|_{\infty} \leq \frac{2 \sqrt{2|S| H^2 a}}{n^{1 / 4}}
$$

も確率$1-|S|^2|A| H \delta_P$以上で成立します。


証明します。まず、$\Delta_h=\max _s\left|V_h^\pi(s)-V_h^{\prime \pi}(s)\right|$とします。このとき、

$$
\begin{aligned}
\Delta_h&=\left|V_h^\pi(s)-V_h^{\prime \pi}(s)\right|=\left|r_\pi(s)+\sum_{s^{\prime}} P_\pi\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)-\left(r_\pi(s)+\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right)\right)\right|\\
& =\left|\sum_{s^{\prime}} P_\pi\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)-\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)+\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)-\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^{\prime \pi}\left(s^{\prime}\right)\right| \\
& \leq \sum_{s^{\prime}} \mid\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right) \mid H+\Delta_{h+1}\right. \\
& \leq|S| H a \frac{1}{\sqrt{n}}+\Delta_{h+1} .
\end{aligned}
$$

が成り立ちます。よって

$$\Delta_h \leq|S| H a \frac{1}{\sqrt{n}}+\Delta_{h+1}$$

が確率$1-|S|^2|A| H \delta_P$以上で成立します。再帰的に展開すると、

$$
\Delta_{H-1}=0 \leq \cdots \leq \Delta_0 \leq|S| H^2 a \frac{1}{\sqrt{n}}
$$
が成り立ちます。よって１つ目が証明できました。

続いて、

$$
\begin{aligned}
\sigma_h^{\pi^2}(s) & =\sigma_h^{\pi^2}(s)-\mathbb{E}^{\prime}\left[\left(V_{h+1}^\pi\left(s_{h+1}\right)-P_\pi^{\prime} V_{h+1}^\pi(s)\right)^2\right]+\mathbb{E}^{\prime}\left[\left(V_{h+1}^\pi\left(s_{h+1}\right)-P_\pi^{\prime} V_{h+1}^\pi(s)\right)^2\right] \\
& \leq \sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^{\pi^2}\left(s^{\prime}\right)-\left[\left(\sum_\pi P_\pi\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)\right)^2-\left(\sum_{s^{\prime}} P_\pi^{\prime}\left(s^{\prime} \mid s\right) V_{h+1}^\pi\left(s^{\prime}\right)\right)^2\right] \\
& +\left[\sqrt{\mathbb{E}^{\prime}\left[\left(V_{h+1}^\pi\left(s_{h+1}\right)-V_1^{\prime \pi}\left(s_1\right)-P_\pi^{\prime}\left(V_{h+1}^\pi-V_{h+1}^{\prime \pi}\right)(s)\right)^2\right]}+\sqrt{\mathbb{E}^{\prime}\left[\left(V_{h+1}^{\prime \pi}\left(s_{h+1}\right)-P_\pi^{\prime}\left(V_{h+1}^{\prime \pi}\right)(s)\right)^2\right]^2},\right.
\end{aligned}
$$

なので、$\operatorname{Var}_{s^{\prime} \sim P_\pi^{\prime}(\cdot \mid s)}\left(V_h^\pi\left(s^{\prime}\right)-V_h^{\prime \pi}\left(s^{\prime}\right)\right) \leq\left\|V_h^\pi-V_h^{\prime \pi}\right\|_{\infty}^2$と合わせると、

$$
\begin{aligned}
\sigma_h^{\pi^2}(s) &\leq \sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^{\pi^2}\left(s^{\prime}\right)\\
& -\left[\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)-P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^\pi\left(s^{\prime}\right)\right]\left[\sum_{s^{\prime}}\left(P_\pi\left(s^{\prime} \mid s\right)+P_\pi^{\prime}\left(s^{\prime} \mid s\right)\right) V_{h+1}^\pi\left(s^{\prime}\right)\right] \\
& +\left(\left\|V_h^\pi-V_h^{\prime \pi}\right\|_{\infty}+\sigma_h^{\prime \pi}(s)\right)^2
\end{aligned}
$$

が確率$1-|S|\delta_P$以上で成り立ちます。よって、１つ目の結果を使うと
$$
\begin{aligned}
\sigma_h^{\pi^2}(s) & \leq\left[\sigma_h^{\prime}(s)+\frac{|S| H^2 a}{\sqrt{n}}\right]^2+\frac{2|S| a H^2}{\sqrt{n}} \leq\left[\sigma_h^{\prime} \pi(s)+\frac{|S| H^2 a}{\sqrt{n}}+\frac{\sqrt{2|S| H^2 a}}{n^{1 / 4}}\right]^2 \\
& \leq\left[\sigma_h^{\prime} \pi(s)+\frac{2 \sqrt{2|S| H^2 a}}{n^{1 / 4}}\right]^2
\end{aligned}
$$

が確率$1-|S|^2|A| H \delta_P$以上で成立します。
これで証明終わりです。


**ステップ６**

* $n \geq 2592|S|^2 H^2 \log 4 / \delta_P$ならば、任意の方策$\pi$について
    * $\left\|V_0^\pi-\tilde{V}_0^\pi\right\|_{\infty} \leq \sqrt{\frac{32|S| H^3}{n}}$ が確率$1-3|S|^2|A| H \delta_P$で成立
    * $\left\|C_{i, 0}^\pi-\tilde{C}_{i, 0}^\pi\right\|_{\infty} \leq \sqrt{\frac{32|S| H^3}{n}}$ が任意の$i\in [N]$について、確率$1-3|S|^2|A| H \delta_P$で成立

を証明します。

まず、上でやったように

$$
\left|P\left(s^{\prime} \mid s, a\right)-\tilde{P}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{8 P\left(s^{\prime} \mid s, a\right)\left(1-\tilde{P}\left(s^{\prime} \mid s, a\right)\right)}{n} \log 4 / \delta_P}+2 \sqrt{2}\left(\frac{\log 4 / \delta_P}{n}\right)^{\frac{3}{4}}+3 \sqrt{2} \frac{\log 4 / \delta_P}{n}
$$

です。$c_1=2 \sqrt{2}\left(\frac{\log 4 / \delta_P}{n}\right)^{\frac{3}{4}}+\frac{3 \sqrt{2} \log 4 / \delta_P}{n}$ and $c_2=\sqrt{\frac{8 \log 4 / \delta_P}{n}}$として、

$$
\begin{aligned}
& \left|V_0^\pi(s)-\tilde{V}_0^\pi(s)\right|=\left|\sum_{h=0}^{H-2} \tilde{P}_\pi^{h-1}\left(P_\pi-\tilde{P}_\pi\right) V_{h+1}^\pi\right|(s) \\
& \leq \sum_{h=0}^{H-2} \tilde{P}_\pi^{h-1}\left|\left(P_\pi-\tilde{P}_\pi\right) V_{h+1}^\pi\right|(s) \leq \sum_{h=0}^{H-2} \tilde{P}_\pi^{h-1}\left(|S| c_1\left\|V_{h+1}^\pi\right\|_{\infty}+c_2 \sqrt{|S|} \sigma_h^\pi\right)(s) \\
& \leq|S| H^2 c_1+c_2 \sqrt{|S|} \sum_{h=0}^{H-1}\left(\tilde{P}_\pi^{h-1} \sigma_h^\pi\right)(s) \\
& \leq|S| H^2 c_1+c_2 \sqrt{|S|} \sum_{h=0}^{H-1}\left(\tilde{P}^{h-1}\left(\tilde{\sigma}_h^\pi+\frac{2^{1.25}|S|^{0.5} H\left(\log 4 / \delta_P\right)^{0.25}}{n^{1 / 4}}\right)(s)\right. \\
& \leq|S| H^2 c_1+c_2 \sqrt{|S| H} \sqrt{\sum_{h=0}^{H-1}\left(\tilde{P}^{h-1} \tilde{\sigma}_h^{\pi^2}\right)(s)}+c_2 H \sqrt{|S|} \frac{2^{1.25}|S|^{0.5} H\left(\log 4 / \delta_P\right)^{0.25}}{n^{1 / 4}} \\
& =\frac{3 \sqrt{2}|S| H^2 \log 4 / \delta_P}{n}+\frac{2 \sqrt{2}|S| H^2\left(\log 4 / \delta_P\right)^{\frac{3}{4}}}{n^{\frac{3}{4}}}+\sqrt{\frac{8|S| H^3 \log 4 / \delta_P}{n}}+\frac{2^{2.75}|S| H^2\left(\log 4 / \delta_P\right)^{\frac{3}{4}}}{n^{\frac{3}{4}}} \\
& \leq \sqrt{32 \frac{|S| H^3}{n}}
\end{aligned}
$$

が成り立ちます。（途中でTotal Variance的なものを使ってるかも。Lemma11参照）


**ステップ７**

あとは全部合体させるだけです。


## Online-CRL アルゴリズム

* 証明はこれに近いみたい：
    * 無限ホライゾン版：[PAC Bounds for Discounted MDPs](https://arxiv.org/abs/1202.3890)
    * 有限ホライゾン版：[Sample Complexity of Episodic Fixed-Horizon Reinforcement Learning](https://arxiv.org/abs/1510.08906)

続いてオンラインRLをする手法を確認しましょう。
Generative modelの場合と似たようなアプローチを取ります。各エピソード$k$で、次のMDPを構築します：

$$
\begin{aligned}
& \mathcal{M}_k:=\left\{M^{\prime}: r^{\prime}(s, a)=r(s, a)
c^{\prime}(i, s, a)=c(i, s, a), H^{\prime}=H, s_0^{\prime}=s_0
\left|P^{\prime}\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq 
\min \left(\sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n(s, a)} \log \frac{4}{\delta_1}}
+\frac{2}{3 n(s, a)} \log \frac{4}{\delta_1}, \sqrt{\frac{\log 4 / \delta_1}{2 n(s, a)}}\right) \forall s, s^{\prime}, a, i\right\},
\end{aligned}
$$

同様に、次の線形計画問題を解きます

$$
\max _{\pi, M^{\prime} \in \mathcal{M}_k} V_0^{\prime \pi}\left(s_0\right) \text { s.t. } C_{i, 0}^{\prime \pi}\left(s_0\right) \leq \bar{C}_i \forall i
$$


\begin{equation}
\begin{aligned}
& \text { 1: Input: accuracy } \epsilon \text { and failure tolerance } \delta \text {. } \\
& \text { 2: Set } k=1, w_{\min }=\frac{\epsilon}{4 H|S|}, U_{\max }=|S|^2|A| m, \delta_1=\frac{\delta}{4(N+1)|S| U_{\max }} \text {. } \\
& \text { 3: Set } m \text { according to }(9) \text { and }(10) \text {. } \\
& \text { 4: Set } n(s, a)=n\left(s^{\prime}, s, a\right)=0 \forall s, s^{\prime} \in S, a \in A \text {. } \\
& \text { 5: while there is }(s, a) \text { with } n(s, a)<|S| m H \text { do } \\
& \text { 6: } \quad \widehat{P}\left(s^{\prime} \mid s, a\right)=\frac{n\left(s^{\prime}, s, a\right)}{n(s, a)} \forall(s, a) \text { with } n(s, a)>0 \text { and } s^{\prime} \in S \text {. } \\
& \text { 7: } \quad \text { Construct } \mathcal{M}_k \text { according to (8). } \\
& \text { 8: } \quad \tilde{\pi}_k=\operatorname{ELP}\left(\mathcal{M}_k\right) \text {. } \\
& \text { 9: } \quad \text { for } t=1, \ldots, H \text { do } \\
& \text { 10:} \quad \quad a_t \sim \tilde{\pi}_k\left(s_t\right), s_{t+1} \sim P\left(\cdot \mid s_t, a_t\right), n\left(s_t, a_t\right)++, n\left(s_{t+1}, s_t, a_t\right)++ \text {. } \\
& \text { 11: } \quad k++
\end{aligned}
\end{equation}

TODO: 元論文ではWhile文でやってるが、これは間違ってる気がする。Lemma12があってる気がしない。for文で書き直そう。

次のサンプル効率を導出していきましょう. 

---

$0 < \epsilon$かつ$\delta <1$のとき、Online CRLアルゴリズムは
$$
\mathbb{P}\left(V_0^{\tilde{\pi}}\left(s_0\right) \geq V_0^{\pi^*}\left(s_0\right)-\epsilon \text { and } C_{i, 0}^{\tilde{\pi}}\left(s_0\right) \leq \bar{C}_i+\epsilon \forall i \in\{1,2, \ldots, N\}\right) \geq 1-\delta
$$
を満たし、必要なサンプル数は$\tilde{O}\left(\frac{|S|^2|A| H^2}{\epsilon^2} \log \frac{N+1}{\delta}\right)$である。

---


### 証明

**ステップ１** $M \in \mathcal{M}_k$が確率$1-\frac{\delta}{2(N+1)}$以上で全ての$k$について成立することを示す。

まずHoeffdingの不等式より
$$
\left|P\left(s^{\prime} \mid s, a\right)-\hat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{\log \left(4 / \delta_1\right)}{2 n(s, a)}}
$$
が確率$1-\delta_1/2$以上で成立。また、Empirical Bernstein（[Empirical Bernstein Bounds and Sample Variance Penalization](https://arxiv.org/abs/0907.3740)）より
$$
\left|P\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n(s, a)} \log \frac{4}{\delta_1}}+\frac{2}{3 n(s, a)} \log \frac{4}{\delta_1}
$$
が確率$1-\delta_1/2$以上で成立。
よって、

$$
\mathbb{P}\left(\left|P\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \min \left\{\sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n(s, a)} \log \frac{4}{\delta_1}}+\frac{2}{3 n(s, a)} \log \frac{4}{\delta_1}, \sqrt{\frac{\log 4 / \delta_1}{2 n(s, a)}}\right\}\right) \geq 1-\delta_1
$$

が成立する。


**ステップ２**

証明のために次の表記を導入します。期待頻度を以下で定義します：

$$
w_k(s, a) :=\sum_{h=0}^{H-1} \mathbb{P}\left(s_h=s, a \sim \tilde{\pi}_k\left(s_h, \cdot, h\right)\right)
=\sum_{h=0}^{H-1} P_{\tilde{\pi}_k}^{h-1} \mathbb{I}\left\{s=\cdot, a \sim \tilde{\pi}_k(s, \cdot, h)\right\}\left(s_0\right)
$$

また、次の「重要度」の概念を導入します：
$$
\iota_k(s, a):=\min \left\{z_j: z_j \geq \frac{w_k(s, a)}{w_{\min }}\right\}
$$
ここで、$w_{\min }:=\frac{\epsilon}{4 H|S|}$、$z_1=0$、$z_j=2^{j-2} \forall j=2,3, \ldots$とします。
$\iota_k(s, a) \in\{0,1,2,4,8,16, \ldots\}$は$w_{\min}$に対して訪問する確率のスケールを表します。
「重要度が高い」$\approx$「$0\sim H$で訪れる回数が高い」ことになります。
また、「認知度」の概念を導入します：
$$
\kappa_k(s, a):=\max \left\{z_i: z_i \leq \frac{n_k(s, a)}{m w_k(s, a)}\right\} \in\{0,1,2,4, \ldots\}
$$
$m$は後で出てくる定数です。
$(s, a)$が訪問された回数を重要度でスケールした量です。「認知度が高い」$\approx$「重要度が低い　もしくは　頻繁に訪れた」ことになります。

$X_k=\left\{(s, a): \iota_k(s, a)>0\right\}$とします。つまり、$X_k$は$0\sim H$で訪れる可能性がある状態と行動の集合です。
また、$\bar{X}_k=S \times A \backslash X_k$とします。つまり訪れる可能性がほぼない状態と行動の集合です。

これを使って、
$$
X_{k, \kappa, \iota}:=\left\{(s, a) \in X_k: \kappa_k(s, a)=\kappa, \iota_k(s, a)=\iota\right\}
$$
を定義します。つまり、$X_{k, \kappa, \iota}$は「重要度が$\iota$で、認知度が$\kappa$な集合」のことになります。

このとき、$X_{k, \kappa, \iota}$の要素を訪れる回数をバウンドします。

まず、$w_\iota=\min \left\{w_k(s, a): \iota_k(s, a)=\iota\right\}$とします。
すると$w_{\iota + 1}= 2 w_\iota$です。

いま、$X_{k, \kappa, \iota}$では$\iota_k(s, a)=\iota$なので、$w_\iota \leq w_k(s, a) \leq 2 w_\iota$です（この表記微妙だな...TODO: 修正しよう）。

同様にして、$\frac{n_k(s, a)}{2 m w_k(s, a)} \leq \kappa \leq \frac{n_k(s, a)}{m w_k(s, a)}$である。

これを使って、

$$
m w_\iota \kappa \leq m w_k(s, a) \kappa \leq n_k(s, a) \leq 2 m w_k(s, a) \kappa \leq 4 m w_\iota \kappa
$$

が成立する。よって、$X_{k, \kappa, \iota}$の要素全てを訪れた回数は$$3|S \times A| m w_\iota \kappa$$以下。
